# Task 1 - Apriori Algorithm for Recommender System (33 points)

**Task Definition:** In this programming assignment, you are required to implement the Apriori algorithm and apply it to mine frequent itemsets for recommendation. You are required to implement the algorithm from scratch by using only native Python libraries and NumPy. For efficiency you will need to convert the items to ids and sort them.

**Input:** The provided input file (`video_games.txt`) contains the favourite video games of 4096 users. Each line in the file corresponds to a user and represents a list of video games the user likes. An example:

*Torchlight 2;Transistor*

In the example above, the corresponding user likes the video games "Torchlight 2" and "Transistor".

**Output:** You need to implement the Apriori algorithm and use it to mine frequent itemsets. Set the relative minimum support to 0.055 and run the algorithm on the 4096 lists of video games. In other words, you need to extract all the itemsets that have an absolute support larger or equal to 226.

In [2]:
# TODO: uncomment the packages you used, please do not import additional non-native packages
# You may change the imports to the following format: from [package] import [class, method, etc.]

import collections
import itertools
import numpy as np

## 1.1 Loading the data and preprocessing (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [3]:
# TODO: read the data from the input file /data/video_games.txt (1 points)

def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()
    return [line.strip().split(';') for line in data]
    
file_path = 'data\\video_games.txt'
transactions = read_data_from_file(file_path)
transactions

[['Greedfall',
  'Code Vein',
  'Path Of Exile',
  'The Outer Worlds',
  'Bastion',
  'Hades',
  'The Outer Worlds',
  "Assassin's Creed Origins",
  'Legend of Grimrock 2',
  'Darkest Dungeon',
  'Pillars of Eternity',
  'Persona 5 Strikers',
  'Transistor'],
 ['Legend of Grimrock 2',
  'The Legend of Zelda: Breath of the Wild',
  'Dying Light 2 Stay Human',
  'The Last Oricru',
  'Darkest Dungeon',
  'Undertale',
  'Dragon Age: Origins'],
 ['Kingdom Hearts 3', 'Torchlight 2'],
 ['Scarlet Nexus',
  'Undertale',
  'Elden Ring',
  'Persona 5 Strikers',
  'Dying Light 2 Stay Human',
  'Soulstice',
  'Tales of Berseria',
  'Dragons Dogma: Dark Arisen',
  'Hades',
  'NieR: Automata',
  'Greedfall',
  'Chrono Trigger',
  'Mass Effect Legendary Edition',
  'The Outer Worlds',
  'Fire Emblem: Three Houses',
  'Weird West'],
 ['Legend of Grimrock 2',
  'The Outer Worlds',
  'Middle-Earth: Shadow of War',
  'Fable II',
  'Scarlet Nexus',
  'Weird West',
  'Bloodborne',
  'Genshin Impact',
  'The

In [158]:
# TODO: determine the unique items and create a dictionary that maps an item to its id using enumerate (1 points)
unique_items = []
for transaction in transactions:
    for item in transaction:
        if item not in unique_items:
            unique_items.append(item)
id_to_item = unique_items
id_to_item  = {i: item for i, item in enumerate(unique_items)}

item_to_id = {item: i for i, item in enumerate(unique_items)}
item_to_id
item_to_id

{'Greedfall': 0,
 'Code Vein': 1,
 'Path Of Exile': 2,
 'The Outer Worlds': 3,
 'Bastion': 4,
 'Hades': 5,
 "Assassin's Creed Origins": 6,
 'Legend of Grimrock 2': 7,
 'Darkest Dungeon': 8,
 'Pillars of Eternity': 9,
 'Persona 5 Strikers': 10,
 'Transistor': 11,
 'The Legend of Zelda: Breath of the Wild': 12,
 'Dying Light 2 Stay Human': 13,
 'The Last Oricru': 14,
 'Undertale': 15,
 'Dragon Age: Origins': 16,
 'Kingdom Hearts 3': 17,
 'Torchlight 2': 18,
 'Scarlet Nexus': 19,
 'Elden Ring': 20,
 'Soulstice': 21,
 'Tales of Berseria': 22,
 'Dragons Dogma: Dark Arisen': 23,
 'NieR: Automata': 24,
 'Chrono Trigger': 25,
 'Mass Effect Legendary Edition': 26,
 'Fire Emblem: Three Houses': 27,
 'Weird West': 28,
 'Middle-Earth: Shadow of War': 29,
 'Fable II': 30,
 'Bloodborne': 31,
 'Genshin Impact': 32,
 'Horizon Zero Dawn': 33,
 'Dragon Age: Inquisition': 34,
 'Crosscode': 35,
 'Kingdom Come: Deliverance': 36,
 'Fallout New Vegas': 37,
 'The Witcher 3: Wild Hunt': 38,
 'Tales of Arise': 

In [159]:
id_to_item

{0: 'Greedfall',
 1: 'Code Vein',
 2: 'Path Of Exile',
 3: 'The Outer Worlds',
 4: 'Bastion',
 5: 'Hades',
 6: "Assassin's Creed Origins",
 7: 'Legend of Grimrock 2',
 8: 'Darkest Dungeon',
 9: 'Pillars of Eternity',
 10: 'Persona 5 Strikers',
 11: 'Transistor',
 12: 'The Legend of Zelda: Breath of the Wild',
 13: 'Dying Light 2 Stay Human',
 14: 'The Last Oricru',
 15: 'Undertale',
 16: 'Dragon Age: Origins',
 17: 'Kingdom Hearts 3',
 18: 'Torchlight 2',
 19: 'Scarlet Nexus',
 20: 'Elden Ring',
 21: 'Soulstice',
 22: 'Tales of Berseria',
 23: 'Dragons Dogma: Dark Arisen',
 24: 'NieR: Automata',
 25: 'Chrono Trigger',
 26: 'Mass Effect Legendary Edition',
 27: 'Fire Emblem: Three Houses',
 28: 'Weird West',
 29: 'Middle-Earth: Shadow of War',
 30: 'Fable II',
 31: 'Bloodborne',
 32: 'Genshin Impact',
 33: 'Horizon Zero Dawn',
 34: 'Dragon Age: Inquisition',
 35: 'Crosscode',
 36: 'Kingdom Come: Deliverance',
 37: 'Fallout New Vegas',
 38: 'The Witcher 3: Wild Hunt',
 39: 'Tales of Aris

In [6]:
mapped_records = []

for record in transactions:
    mapped_record = [item_to_id[item] for item in record]
    mapped_record.sort()
    mapped_records.append(mapped_record)

print(mapped_records)

[[0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11], [7, 8, 12, 13, 14, 15, 16], [17, 18], [0, 3, 5, 10, 13, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], [3, 3, 7, 19, 28, 29, 30, 31, 32], [8, 28, 33, 34], [2, 8, 20, 21, 35], [0, 5, 9, 27, 28], [0, 2, 3, 8, 11, 17, 20, 22, 25, 26, 33, 34, 36], [0, 1, 3, 4, 8, 10, 16, 22, 23, 31, 32, 34, 35, 37], [7, 10, 14, 24, 30, 33, 38], [18, 25, 39], [2, 15, 18, 34, 36, 40, 41], [1, 35, 39, 42], [15, 16, 26], [0, 7, 15, 22, 41, 43], [4, 7, 8, 15, 20, 25, 28, 36, 44, 45], [3, 4, 5, 6, 7, 10, 11, 19, 25, 37, 44, 46], [0, 14, 18, 22, 23, 24, 32, 38, 40, 44], [0, 3, 3, 5, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 27, 29, 31, 32, 35, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48], [5, 6, 8, 19, 22, 34, 38, 43], [0, 3, 3, 6, 8, 10, 11, 15, 20, 34, 37, 46], [4, 9, 10, 12, 19, 26, 29, 32, 33, 39, 41], [5, 9, 12, 14, 24, 25, 28, 37, 47], [6, 10, 11, 22, 25, 26, 30, 32, 45], [1, 14, 20, 27, 42, 45], [3, 8, 9, 11, 15, 31, 41], [3, 5, 12, 28], [0, 1, 3, 6, 

## 1.2 Apriori algorithm (21 points)
### A) Determine the frequent 1 itemsets (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [7]:
from collections import Counter

l1_items = Counter()

for record in mapped_records:
    l1_items.update(record)

In [144]:
# TODO: filter out the frequent length-1 itemsets with their support (1 point)
min_support = 200  # Set the minimum support threshold as desired

frequent_1_itemsets = [item for item, support in l1_items.items() if support >= min_support]

print(frequent_1_itemsets)

# TODO: save the length-1 frequent items to frequent_items with their support (1 points)
# Hint: Convert the itemsets to tuples or sets so that you can use them as keys in a dictionary.
frequent_items = {}

for item, support in l1_items.items():
    if support >= min_support:
        frequent_items[item] = support

print(frequent_items)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
{0: 303, 1: 1556, 2: 378, 3: 620, 4: 373, 5: 348, 6: 1035, 7: 246, 8: 213, 9: 269, 10: 339, 11: 758, 12: 298, 13: 1489, 14: 251, 15: 248, 16: 238, 17: 415, 18: 980, 19: 774, 20: 1648, 21: 232, 22: 684, 23: 680, 24: 218, 25: 228, 26: 318, 27: 264, 28: 250, 29: 667, 30: 248, 31: 379, 32: 1046, 33: 340, 34: 329, 35: 363, 36: 753, 37: 389, 38: 303, 39: 390, 40: 431, 41: 1539, 42: 1654, 43: 319, 44: 633, 45: 1383, 46: 1052, 47: 1104, 48: 230}


### B) Determine the frequent n itemsets (15 points)
**Task:** Solve the tasks explained in the TODOs and comments.


In [130]:
def apriori_gen(itemsets):
    candidates = []
    k = len(itemsets[0]) + 1  # k is the length of the itemsets to be generated

    # Generate candidates
    for i in range(len(itemsets)):
        for j in range(i + 1, len(itemsets)):
            # Joining step
            if itemsets[i][:k-2] == itemsets[j][:k-2]:
                candidate = itemsets[i] + [itemsets[j][-1]]
                candidates.append(candidate)

    # Prune candidates
    pruned_candidates = []
    for candidate in candidates:
        is_subsets_frequent = True
        # Generate all possible subsets of candidate with length k-1
        subsets = [candidate[:i] + candidate[i+1:] for i in range(k)]
        for subset in subsets:
            # Check if the subset is in the itemsets
            if subset not in itemsets:
                is_subsets_frequent = False
                break
        if is_subsets_frequent:
            pruned_candidates.append(candidate)

    return pruned_candidates

In [131]:
def calculate_support(itemset):
    count = 0

    for record in mapped_records:
        if set(itemset).issubset(record):
            count += 1

    return count

In [145]:
# # Set the initial candidates for length-2 itemsets
candidates = [[item] for item in frequent_1_itemsets]
print(candidates)
# Loop until there are no more candidates to generate
while candidates:
    new_candidates = apriori_gen(candidates)
    supports = map(calculate_support, new_candidates)

    # Filter out the frequent candidates
    frequent_new_candidates = [candidate for candidate, support in zip(new_candidates, supports) if support >= min_support]
    print(frequent_new_candidates)
    # Add the frequent candidates to frequent_items
    for candidate in frequent_new_candidates:
        frequent_items[tuple(candidate)] = calculate_support(candidate)

    # Replace candidates with the new ones
    candidates = frequent_new_candidates

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48]]
[[1, 6], [1, 11], [1, 13], [1, 18], [1, 19], [1, 20], [1, 22], [1, 23], [1, 29], [1, 32], [1, 36], [1, 41], [1, 42], [1, 44], [1, 45], [1, 46], [1, 47], [6, 11], [6, 13], [6, 18], [6, 19], [6, 20], [6, 22], [6, 23], [6, 29], [6, 32], [6, 36], [6, 41], [6, 42], [6, 44], [6, 45], [6, 46], [6, 47], [11, 13], [11, 18], [11, 19], [11, 20], [11, 22], [11, 23], [11, 29], [11, 32], [11, 36], [11, 41], [11, 42], [11, 44], [11, 45], [11, 46], [11, 47], [13, 18], [13, 19], [13, 20], [13, 22], [13, 23], [13, 29], [13, 32], [13, 36], [13, 41], [13, 42], [13, 44], [13, 45], [13, 46], [13, 47], [18, 19], [18, 20], [18, 22], [18, 23], [18, 29], [18, 32], [18, 36], [18, 41], [18, 42], [18, 44], [18, 45], [18, 46], [18, 47]

### C) Save your results (3 points)

**Task:** Save all the frequent itemsets along with their absolute supports into a text file named “patterns.txt” and place it in the root of your zip file. Every line corresponds to exactly one frequent itemset and should be in the following format:

*support:video_game_1;video_game_2;video_game_3;...*

For example, suppose an itemset (Fallout 4;Genshin Impact) has an absolute support 520, then the line corresponding to this frequent itemset in `patterns.txt` should be:

*520:Fallout 4;Genshin Impact*

In [143]:
frequent_items.items()

dict_items([(0, 303), (1, 1556), (2, 378), (3, 620), (4, 373), (5, 348), (6, 1035), (10, 339), (11, 758), (13, 1489), (17, 415), (18, 980), (19, 774), (20, 1648), (22, 684), (23, 680), (26, 318), (29, 667), (31, 379), (32, 1046), (33, 340), (34, 329), (35, 363), (36, 753), (37, 389), (38, 303), (39, 390), (40, 431), (41, 1539), (42, 1654), (43, 319), (44, 633), (45, 1383), (46, 1052), (47, 1104), ((1, 6), 544), ((1, 11), 372), ((1, 13), 800), ((1, 18), 506), ((1, 19), 352), ((1, 20), 893), ((1, 22), 327), ((1, 23), 318), ((1, 29), 316), ((1, 32), 522), ((1, 36), 355), ((1, 41), 846), ((1, 42), 889), ((1, 44), 303), ((1, 45), 761), ((1, 46), 520), ((1, 47), 556), ((6, 11), 306), ((6, 13), 505), ((6, 18), 412), ((6, 19), 305), ((6, 20), 572), ((6, 32), 441), ((6, 41), 519), ((6, 42), 540), ((6, 45), 511), ((6, 46), 453), ((6, 47), 455), ((11, 13), 351), ((11, 18), 302), ((11, 20), 373), ((11, 32), 302), ((11, 41), 355), ((11, 42), 363), ((11, 45), 314), ((11, 46), 317), ((11, 47), 323), 

In [154]:
id_to_item[1]

KeyError: 1

In [160]:
file_path = "patterns.txt"

with open(file_path, 'w') as file:
    for itemset, support in frequent_items.items():
        line = f'{support}:'
        if (type(itemset) != int):
            for i in itemset:
                line+= f'{id_to_item[i]};'
        else:
            # print(id_to_item[itemset])
            line+= f'{id_to_item[itemset]};'

        file.write(line + "\n")

## 1.3 Recommendation (9 points)

**Task:** Imagine you should recommend a video game to a user. You know that the user likes the video games "Elden Ring" and "Scarlet Nexus". Based on the result of the Apriori algorithm, implement an algorithm that gives a recommendation for this user by maximizing the confidence that the user will like the game. (6 points)

**Report:** Explain your choice and report the confidence score for your recommendation. (3 points)


In [128]:
frequent_items.keys()


dict_keys([0, 1, 2, 3, 4, 5, 6, 10, 11, 13, 17, 18, 19, 20, 22, 23, 26, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, (1, 6), (1, 11), (1, 13), (1, 18), (1, 19), (1, 20), (1, 22), (1, 23), (1, 29), (1, 32), (1, 36), (1, 41), (1, 42), (1, 44), (1, 45), (1, 46), (1, 47), (6, 11), (6, 13), (6, 18), (6, 19), (6, 20), (6, 32), (6, 41), (6, 42), (6, 45), (6, 46), (6, 47), (11, 13), (11, 18), (11, 20), (11, 32), (11, 41), (11, 42), (11, 45), (11, 46), (11, 47), (13, 18), (13, 19), (13, 20), (13, 23), (13, 32), (13, 36), (13, 41), (13, 42), (13, 45), (13, 46), (13, 47), (18, 19), (18, 20), (18, 32), (18, 36), (18, 41), (18, 42), (18, 45), (18, 46), (18, 47), (19, 20), (19, 41), (19, 42), (19, 45), (19, 46), (19, 47), (20, 22), (20, 23), (20, 29), (20, 32), (20, 36), (20, 41), (20, 42), (20, 44), (20, 45), (20, 46), (20, 47), (22, 41), (22, 42), (23, 41), (23, 42), (23, 45), (29, 41), (29, 42), (32, 36), (32, 41), (32, 42), (32, 45), (32, 46), (32, 47), (36, 41), (36, 

In [167]:
target_games = [ item_to_id["Scarlet Nexus"],item_to_id["Elden Ring"]]
recommendation = None
max_confidence = 0.0
target_games_tuple = tuple(target_games)
# Iterate over the frequent itemsets
for itemset, support in frequent_items.items():
    if isinstance(itemset, int):
        itemset = [itemset]
    elif isinstance(itemset, tuple):
        itemset = list(itemset)

    if set(target_games).issubset(itemset):
        # Calculate the confidence score
        confidence = support/frequent_items[target_games_tuple]
        if confidence == 1:
            confidence = 0

        # Update the recommendation if the confidence is higher
        if confidence > max_confidence:
            max_confidence = confidence
            recommendation = itemset

# Convert item IDs back to item names for the recommendation
if recommendation != None:
    recommendation_names = [id_to_item[item_id] for item_id in recommendation]
    print("Recommendation:", recommendation_names)
    print("Confidence:", max_confidence)

# Print the recommendation and confidence score



Recommendation: ['Scarlet Nexus', 'Elden Ring', 'Cyberpunk 2077']
Confidence: 0.6587301587301587
